In [2]:
model = model.to('cuda')
pip install transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import load_dataset, load_metric

2024-08-08 05:58:21.676691: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 05:58:21.676841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 05:58:21.819945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

print(dataset)

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [5]:
# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=9)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Tokenize the inputs and align the labels with tokens
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Create a data collator for dynamic padding
data_collator = DataCollatorForTokenClassification(tokenizer)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
# Load the pre-trained BERT model
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(dataset['train'].features['ner_tags'].feature.names))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.172800,0.047661


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=878, training_loss=0.12225914218702728, metrics={'train_runtime': 354.3752, 'train_samples_per_second': 39.622, 'train_steps_per_second': 2.478, 'total_flos': 351240792638148.0, 'train_loss': 0.12225914218702728, 'epoch': 1.0})

In [13]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.04766146093606949, 'eval_runtime': 14.3507, 'eval_samples_per_second': 226.47, 'eval_steps_per_second': 14.215, 'epoch': 1.0}


In [18]:
# Example sentence for NER
example_sentence = "Sundharess is studying Msc in Artificial intelligence in christ University"

# Tokenize the example sentence
inputs = tokenizer(example_sentence, return_tensors="pt")

# Get model predictions
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)

# Decode the predictions
predicted_labels = [model.config.id2label[label_id.item()] for label_id in predictions[0]]

print(f"Sentence: {example_sentence}")
print(f"Predicted labels: {predicted_labels}")


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [17]:
model = model.to('cuda')


In [19]:
inputs = {key: value.to('cuda') for key, value in inputs.items()}


In [20]:
from transformers import BertTokenizer, BertForTokenClassification

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')

# Move model to GPU
model = model.to('cuda')

# Example input
example_sentence = "Sundharess is studying Msc in Artificial intelligence in christ University"
inputs = tokenizer(example_sentence, return_tensors="pt")

# Move inputs to GPU
inputs = {key: value.to('cuda') for key, value in inputs.items()}

# Get model predictions
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)

# Example list of labels (adjust according to your model's label set)
labels = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"]

# Get the predicted indices (assuming predictions is a tensor)
predicted_indices = predictions[0].tolist()  # Convert tensor to list

# Convert indices to labels
predicted_labels = [labels[idx] for idx in predicted_indices]

# Print or use the predicted ++++3

+3labels
print(predicted_labels)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['O', 'B-PER', 'B-PER', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'B-PER']


In [23]:
# Example input
example_sentence = "Barack Obama was born in Hawaii."
inputs = tokenizer(example_sentence, return_tensors="pt")

# Move inputs to GPU
inputs = {key: value.to('cuda') for key, value in inputs.items()}

# Get model predictions
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)

# Example list of labels (adjust according to your model's label set)
labels = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"]

# Get the predicted indices (assuming predictions is a tensor)
predicted_indices = predictions[0].tolist()  # Convert tensor to list

# Convert indices to labels
predicted_labels = [labels[idx] for idx in predicted_indices]


In [24]:
print(predicted_labels)


['O', 'B-PER', 'B-PER', 'O', 'O', 'B-PER', 'B-PER', 'O', 'B-PER']


In [25]:
from transformers import BertTokenizer, BertForTokenClassification

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')

# Move model to GPU
model = model.to('cuda')

# Example input
example_sentence = "karhikeyan studies in christ University"
inputs = tokenizer(example_sentence, return_tensors="pt")

# Move inputs to GPU
inputs = {key: value.to('cuda') for key, value in inputs.items()}

# Get model predictions
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)

# Example list of labels (adjust according to your model's label set)
labels = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"]

# Get the predicted indices (assuming predictions is a tensor)
predicted_indices = predictions[0].tolist()  # Convert tensor to list

# Convert indices to labels
predicted_labels = [labels[idx] for idx in predicted_indices]

# Print or use the predicted ++++3

print(predicted_labels)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['B-PER', 'B-PER', 'O', 'B-PER', 'O', 'B-PER', 'O', 'O', 'B-PER', 'O']
